# 📊 Step 1: データ管理の基礎

機械学習プロジェクトにおいて、**データ管理**は最も重要な基礎スキルの一つです。

## 🎯 このノートブックで学ぶこと
- ローカルデータの作成と確認
- S3への効率的なデータアップロード
- S3からのデータダウンロード
- データのバージョン管理
- 実際のプロジェクトでのベストプラクティス

## ⏱️ 実行時間: 約5分

## 1. 環境設定とライブラリ

In [ ]:
import sagemaker
import boto3
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime
from sagemaker import get_execution_role

# SageMaker設定
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = boto3.Session().region_name
bucket = sagemaker_session.default_bucket()

print("🔧 データ管理環境の設定")
print(f"📍 Region: {region}")
print(f"🪣 S3 Bucket: {bucket}")
print(f"👤 Role: {role.split('/')[-1]}")
print(f"📅 現在時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. ローカルデータの作成と確認

### 📚 学習ポイント
- データの生成と保存
- ファイルサイズとデータ形式の確認
- データ品質のチェック

In [ ]:
# データが存在しない場合は生成
if not os.path.exists('./data/train_lecture.csv'):
    print("📊 講義用データセットを生成中...")
    import subprocess
    result = subprocess.run(['python3', './data/create_lecture_dataset.py'], 
                          cwd='./data', capture_output=True, text=True)
    print(result.stdout)
else:
    print("✅ データセットは既に存在します")

# ローカルデータの詳細確認
print("\n📋 ローカルデータの詳細情報:")
data_files = ['train_lecture.csv', 'validation_lecture.csv', 'test_lecture.csv']

total_size = 0
for file_name in data_files:
    file_path = f'./data/{file_name}'
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path)
        total_size += file_size
        
        # データを読み込んで基本情報を表示
        df = pd.read_csv(file_path)
        
        print(f"\n📄 {file_name}:")
        print(f"  📏 ファイルサイズ: {file_size/1024:.1f} KB")
        print(f"  📊 データ形状: {df.shape}")
        print(f"  🔍 欠損値: {df.isnull().sum().sum()}個")
        print(f"  📈 クラス分布: {dict(df['target'].value_counts().sort_index())}")
        
        # メモリ使用量
        memory_usage = df.memory_usage(deep=True).sum()
        print(f"  💾 メモリ使用量: {memory_usage/1024:.1f} KB")

print(f"\n📦 総データサイズ: {total_size/1024:.1f} KB")
print(f"💡 データ管理のポイント: 小さなデータでも構造化して管理することが重要")

## 3. S3へのデータアップロード

### 📚 学習ポイント
- S3の階層構造の設計
- データのバージョン管理
- アップロード時間の測定
- メタデータの付与

In [ ]:
# S3のデータ構造を設計
project_name = 'ml-lecture'
version = datetime.now().strftime('%Y%m%d-%H%M%S')
s3_prefix = f'{project_name}/data/v{version}'

print(f"🗂️ S3データ構造の設計:")
print(f"  📁 プロジェクト: {project_name}")
print(f"  🏷️ バージョン: v{version}")
print(f"  📍 S3パス: s3://{bucket}/{s3_prefix}/")

# データアップロードの実行
print(f"\n📤 S3へのデータアップロード開始...")
upload_start_time = time.time()

s3_paths = {}
upload_info = []

for file_name in data_files:
    local_path = f'./data/{file_name}'
    
    if os.path.exists(local_path):
        print(f"  📤 {file_name} をアップロード中...")
        
        # ファイル別のアップロード時間測定
        file_start_time = time.time()
        
        # データタイプに応じたS3パスを設定
        data_type = file_name.replace('_lecture.csv', '')
        s3_path = sagemaker_session.upload_data(
            path=local_path,
            bucket=bucket,
            key_prefix=f'{s3_prefix}/{data_type}'
        )
        
        file_upload_time = time.time() - file_start_time
        file_size = os.path.getsize(local_path)
        
        s3_paths[data_type] = s3_path
        upload_info.append({
            'file': file_name,
            'size_kb': file_size/1024,
            'upload_time': file_upload_time,
            's3_path': s3_path
        })
        
        print(f"    ✅ 完了 ({file_upload_time:.2f}秒, {file_size/1024:.1f}KB)")

total_upload_time = time.time() - upload_start_time

print(f"\n📊 アップロード結果サマリー:")
print(f"  ⏱️ 総アップロード時間: {total_upload_time:.2f}秒")
print(f"  📦 総ファイル数: {len(upload_info)}個")
print(f"  📏 総データサイズ: {sum([info['size_kb'] for info in upload_info]):.1f}KB")
print(f"  🚀 平均アップロード速度: {sum([info['size_kb'] for info in upload_info])/total_upload_time:.1f}KB/秒")

# S3パスの保存（後で使用）
print(f"\n🔗 S3データパス:")
for data_type, path in s3_paths.items():
    print(f"  {data_type}: {path}")

## 4. S3からのデータダウンロード

### 📚 学習ポイント
- S3からの効率的なデータ取得
- ダウンロード vs 直接読み込みの使い分け
- データ整合性の確認

In [ ]:
# S3からのデータダウンロードテスト
print("📥 S3からのデータダウンロードテスト:")

# 一時的なダウンロードディレクトリを作成
download_dir = './temp_download'
os.makedirs(download_dir, exist_ok=True)

download_start_time = time.time()

# 方法1: sagemaker.Session.download_data()を使用
print("\n📥 方法1: SageMaker Session経由でダウンロード")
for data_type, s3_path in s3_paths.items():
    print(f"  📥 {data_type} をダウンロード中...")
    
    file_start_time = time.time()
    
    # S3パスからバケット名とキーを抽出
    s3_parts = s3_path.replace('s3://', '').split('/')
    bucket_name = s3_parts[0]
    s3_key = '/'.join(s3_parts[1:])
    
    # ダウンロード実行
    local_download_path = sagemaker_session.download_data(
        path=download_dir,
        bucket=bucket_name,
        key_prefix=s3_key.rsplit('/', 1)[0]  # ファイル名を除いたパス
    )
    
    download_time = time.time() - file_start_time
    print(f"    ✅ 完了 ({download_time:.2f}秒)")
    print(f"    📁 ローカルパス: {local_download_path}")

# 方法2: pandas.read_csv()で直接読み込み
print(f"\n📥 方法2: pandasで直接S3から読み込み")
direct_read_start = time.time()

# 訓練データを直接読み込み
train_s3_path = s3_paths['train']
df_from_s3 = pd.read_csv(train_s3_path)

direct_read_time = time.time() - direct_read_start
print(f"  ✅ 直接読み込み完了 ({direct_read_time:.2f}秒)")
print(f"  📊 データ形状: {df_from_s3.shape}")

total_download_time = time.time() - download_start_time

print(f"\n📊 ダウンロード結果比較:")
print(f"  📥 ダウンロード方式: {total_download_time:.2f}秒")
print(f"  📖 直接読み込み方式: {direct_read_time:.2f}秒")
print(f"  💡 推奨: 小さなデータは直接読み込み、大きなデータはダウンロード")

## 5. データ整合性の確認

### 📚 学習ポイント
- アップロード・ダウンロード後のデータ検証
- ハッシュ値による整合性チェック
- データ品質の確認

In [ ]:
import hashlib

print("🔍 データ整合性の確認:")

# 元のデータを読み込み
original_df = pd.read_csv('./data/train_lecture.csv')

# S3から読み込んだデータと比較
print(f"\n📊 データ比較:")
print(f"  📄 元データ形状: {original_df.shape}")
print(f"  📄 S3データ形状: {df_from_s3.shape}")
print(f"  ✅ 形状一致: {original_df.shape == df_from_s3.shape}")

# データ内容の比較
data_equal = original_df.equals(df_from_s3)
print(f"  ✅ データ内容一致: {data_equal}")

if not data_equal:
    # 差分の詳細確認
    diff_mask = (original_df != df_from_s3).any(axis=1)
    diff_count = diff_mask.sum()
    print(f"  ⚠️ 差分のある行数: {diff_count}")

# ハッシュ値による検証
def calculate_dataframe_hash(df):
    """DataFrameのハッシュ値を計算"""
    return hashlib.md5(pd.util.hash_pandas_object(df, index=True).values).hexdigest()

original_hash = calculate_dataframe_hash(original_df)
s3_hash = calculate_dataframe_hash(df_from_s3)

print(f"\n🔐 ハッシュ値による検証:")
print(f"  📄 元データハッシュ: {original_hash[:16]}...")
print(f"  📄 S3データハッシュ: {s3_hash[:16]}...")
print(f"  ✅ ハッシュ一致: {original_hash == s3_hash}")

# データ品質チェック
print(f"\n🔍 データ品質チェック:")
quality_checks = {
    '欠損値なし': df_from_s3.isnull().sum().sum() == 0,
    '重複行なし': df_from_s3.duplicated().sum() == 0,
    'ターゲット値正常': df_from_s3['target'].isin([0, 1, 2]).all(),
    '数値列正常': df_from_s3.select_dtypes(include=[np.number]).notna().all().all()
}

for check_name, result in quality_checks.items():
    status = "✅" if result else "❌"
    print(f"  {status} {check_name}: {result}")

print(f"\n🎉 データ管理の基礎完了！")
print(f"💡 次のステップ: このS3データを使って機械学習を実行します")

## 6. データ管理のベストプラクティス

### 📚 実際のプロジェクトで重要なポイント

In [ ]:
print("📋 データ管理のベストプラクティス:")

best_practices = {
    "🗂️ 構造化された命名規則": {
        "説明": "プロジェクト/データタイプ/バージョンの階層構造",
        "例": f"s3://{bucket}/{project_name}/data/v{version}/train/",
        "メリット": "データの発見・管理が容易"
    },
    "🏷️ バージョン管理": {
        "説明": "タイムスタンプまたはセマンティックバージョニング",
        "例": f"v{version} または v1.0.0",
        "メリット": "データの変更履歴を追跡可能"
    },
    "🔍 データ検証": {
        "説明": "アップロード後の整合性チェック",
        "例": "ハッシュ値比較、形状確認、品質チェック",
        "メリット": "データ破損の早期発見"
    },
    "📊 メタデータ管理": {
        "説明": "データの説明、作成日時、サイズ等の記録",
        "例": "data_catalog.json, README.md",
        "メリット": "データの理解と再利用が容易"
    },
    "🔐 アクセス制御": {
        "説明": "適切な権限設定とセキュリティ",
        "例": "IAMロール、バケットポリシー",
        "メリット": "データの安全性確保"
    }
}

for practice, details in best_practices.items():
    print(f"\n{practice}")
    print(f"  📝 説明: {details['説明']}")
    print(f"  💡 例: {details['例']}")
    print(f"  ✨ メリット: {details['メリット']}")

print(f"\n🎯 今回学んだデータ管理スキル:")
skills = [
    "ローカルデータの作成と品質確認",
    "S3への効率的なアップロード",
    "S3からの柔軟なデータ取得",
    "データ整合性の検証",
    "バージョン管理の実践"
]

for i, skill in enumerate(skills, 1):
    print(f"  {i}. ✅ {skill}")

print(f"\n🚀 次のノートブック: 02_script_mode_training.ipynb")
print(f"   今回作成したS3データを使ってScript Modeでの機械学習を実行します！")

## 7. クリーンアップ

In [ ]:
# 一時ファイルのクリーンアップ
import shutil

if os.path.exists(download_dir):
    shutil.rmtree(download_dir)
    print(f"🗑️ 一時ダウンロードディレクトリを削除: {download_dir}")

print(f"\n✅ データ管理の基礎学習完了！")
print(f"📊 作成されたS3データ:")
for data_type, path in s3_paths.items():
    print(f"  {data_type}: {path}")
    
# 次のノートブック用にS3パスを保存
s3_paths_file = 's3_data_paths.json'
import json
with open(s3_paths_file, 'w') as f:
    json.dump(s3_paths, f, indent=2)
    
print(f"\n💾 S3パス情報を保存: {s3_paths_file}")
print(f"🎯 次のステップ: 02_script_mode_training.ipynb を実行してください")